In [ ]:
import os
from datasets import load_dataset, Dataset, DatasetDict
import numpy as np
import pandas as pd
from datetime import datetime
from transformers import (
    TimeSeriesTransformerConfig, TimeSeriesTransformerForPrediction,
    InformerConfig, InformerForPrediction
)

os.chdir("../")

from src.data import HF_Dataset
from src.conf import HF_Dataset_Params, TS_Transformer_Params
from src.utils import get_split_limit, transform_start_field
from src.models import TS_Transformer

In [ ]:
freq = "1H"

prediction_length = 12

In [ ]:
params = HF_Dataset_Params(
    # from DB
    [datetime(2023, 4, 12, 12, 0, 0), datetime(2023, 4, 12, 12, 0, 0)], 
    [[i for i in range(100)], [-i/10 for i in range(100)]],
    [[1], [2]],
    [None, None],
    # from UI
    freq,
    prediction_length
)
custom_ts = HF_Dataset(params)

In [ ]:
custom_ts.getDatasetDict()

In [ ]:
train_dataset = custom_ts.getFormattedDatasetDict()["train"]

test_dataset = custom_ts.getFormattedDatasetDict()["test"]

In [ ]:
custom_ts.getFeaturesStats()

# Test Transformers

In [ ]:
transformer_params = TS_Transformer_Params(
    prediction_length, freq, custom_ts.getFeaturesStats(),
    [InformerConfig, InformerForPrediction],
    4, [2], 4, 32, 0.3
)

In [ ]:
transformer = TS_Transformer(transformer_params)

transformer._model()

In [ ]:
config = transformer.config

In [ ]:
model = transformer.model

In [ ]:
train_dataloader, test_dataloader = transformer._transformation(train_dataset, test_dataset)

In [ ]:
train_dataloader

In [ ]:
batch = next(iter(train_dataloader))
for k, v in batch.items():
    print(k, v.shape, v.type())